In [1]:
%run ./read_file.ipynb 

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-76ed3ecb-3b7a-424c-a346-96c2f470ff23;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (830ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (28ms)
:: resolution report :: resolve 693ms :: artifacts dl 860ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	------

23/03/12 14:15:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/12 14:15:13 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
def explode_and_drop(df_final):
        
    df_final = df_final.withColumn("parent_objectid", split(df_final["parent"], "\^")[0])
    df_final = df_final.withColumn("parent_timestamp", split(df_final["parent"],"\^")[1])
    df_final = df_final.withColumn("parent_object", split(df_final["parent"], "\^")[2])
    df_final = df_final.withColumn("parent_parent_image_path", split(df_final["parent"], "\^")[3])
    df_final = df_final.withColumn("parent_image_path", split(df_final["parent"], "\^")[4])
    df_final = df_final.withColumn("parent_USER", split(df_final["parent"], "\^")[5])

    df_final = df_final.withColumn("child_objectid", split(df_final["child"], "\^")[0])
    df_final = df_final.withColumn("child_timestamp", split(df_final["child"], "\^")[1])
    df_final = df_final.withColumn("child_object", split(df_final["child"], "\^")[2])
    df_final = df_final.withColumn("child_parent_image_path", split(df_final["child"], "\^")[3])
    df_final = df_final.withColumn("child_image_path", split(df_final["child"], "\^")[4])
    df_final = df_final.withColumn("child_USER", split(df_final["child"], "\^")[5])
    
    df_final = df_final.drop("parent","child")
    
    return df_final

In [4]:
def build_adjaceny(df, dataset = 'ecar'):
    
    start_time = time.time()
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()
    
    print(len(col_actID))

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    print("length actIDs: "+str(len(actIDs)))
    print("elapsed time: "+ str(time.time() - start_time))
    #create ref dict
    dict_node_edge_events = {}
    count = 0 
    
    print(actIDs[0])
    
    #for all actors in actIDs
    for aID in actIDs: 
        
        #I think we can pull parent time here and record this in the dictionary somehow
        birth_event = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()
        
        if len(birth_event) > 0: 
            str_first_id = birth_event[0]['id']

            #now add this to the dictionary as a key
            first_event = str(aID+'^'  
                              +str(birth_event[0]['timestamp'])+"^" 
                              +str(birth_event[0]['object'])+"^" 
                              +str(birth_event[0]['parent_image_path'])+"^" 
                              +str(birth_event[0]['image_path'])+'^USER_placeholder')
            
            dict_node_edge_events[first_event] = []

            #now add all the events where the actorID is present but not the objectID, group by unique object ID
            #and pull out the first event - and I checked and believe the sorting holds through grouping
            col_select_children_events = df.where((col("actorID") == aID) & \
                     (col("objectID") != aID)).groupBy("objectID").agg(first("timestamp"),
                                                                      first('object'),
                                                                       first('parent_image_path'),
                                                                       first('image_path')).collect() 

            #add children objects into dictioinary 
            dict_node_edge_events[first_event] = [str(row["objectID"]+'^'
                                                      +str(row['first(timestamp)'])+'^'
                                                      +str(row['first(object)'])+'^'
                                                      +str(row['first(parent_image_path)'])+'^'
                                                      +str(row['first(image_path)'])+'^'
                                                      +'^USER_placeholder') 
                                                  for row in col_select_children_events]
        else:
            count += 1
    
    print("no birth event for " + str(count)+" events.")
    
    # create a list of tuples from the dictionary
    data_list = list(dict_node_edge_events.items())

    # convert the list of tuples to a PySpark RDD
    data_rdd = spark.sparkContext.parallelize(data_list)

    # create a PySpark DataFrame from the RDD
    df_final = data_rdd.toDF()
    
    #explode list into column, keeping position for some sort of rank just in case
    df_final_exp = df_final.select("_1",posexplode("_2"))
    df_final_exp_col = df_final_exp.selectExpr("_1 as parent", "pos as order", "col as child")
    
    print("elapsed time 2: "+ str(time.time() - start_time))
    
    df_final_exp_col = explode_and_drop(df_final_exp_col)
    
    return df_final_exp_col